# BEE 4750 DC Transportation Optimization

#### Group Members: Maya Zamor, MacKenzie Fountain, Noah Hartman, Raashid Husain

Our project will minimize an objective function that calculates the net carbon dioxide emissions produced by tailpipe emissions from various sources of transportation; internal combustion engines (ICEs), electric vehicles (EVs), and buses. The scope of this project aims to include the CO2 emissions from one city. During our preliminary meeting, our team discussed using Ithaca as a model. Decision variables include the number of electric vehicles, gas vehicles, and buses. Constraints include the city’s capacity of gas vehicles and buses, the city’s EV infrastructure capacity, and any emissions standards/thresholds set by the EPA and/or NYSDEC. We chose this topic since we were interested in the cross-section of transportation and air pollution as the shift to electric vehicles expands.


In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")
import Pkg; Pkg.add("DataFrames")
import Pkg; Pkg.add("GraphRecipes")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("Measures")
import Pkg; Pkg.add("MarkdownTables")

In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables




#### Average Distance Traveled per Mode of Transportation

|   Transportation   | Distance (miles) |
|:--------:|:---------------|
|    Gas   |       1      |
| Electric |       2     |
|    Bus   |       3     |
|   Train  |       4     |

.

|   Transportation   | CO2 emissions (lb/yr) | CO2 per Mile (g/mile)| 
|:--------:|:--------------------------:|:---------------------:|
|    Gas   |            350             |         0.xx          |
| Electric |            280             |         0.xx          |
|    Bus   |            390             |         0.xx          |
|   Train  |            390             |         0.xx          |

.

|   Transportation   | Cost ($/year) |
|:--------:|:---------------|
|    Gas   |       1      |
| Electric |       2     |
|    Bus   |       3     |
|   Train  |       4     |

In [ ]:
Transportation = ["Gas", "Electric", "Bus", "Train"]

CO2emissions=[23334,2254,1,1];
#Amount of CO2 emmissions per year correspoinsing to each mode of transportation; in units of lbs/year

CO2_per_Mile=[400,125,1,1];
#Amount of CO2 emmissions per mile correspoinsing to each mode of transportation; in units of grams/mile

Capacity_transportation=[112,1211,1571,1];
#capacity of each mode of transportation; Gas stations, EV Charging ports, Total amount of busses, Total amount of trains

commuters=552683;
#population of DC residents who are over 18 years old

max_fleet_size=

max_budget=

target_EVs=

In [ ]:
m = DC_Transportation_Model(HiGHS.Optimizer)

@variable(m, G >= 0)
@variable(m, E >= 0)
@variable(m, B >= 0)
@variable(m, T >= 0)

@objective(m, Min, ((23334+Dg)G+(2254+De)E+(Eb+Db)B+(Et+Dt)T))

@constraint(m, (G+E+B+T >= commuters)
@constraint(m, (G+E+B+T <= max_fleet_size)
@constraint(m, (Cg*G+Ce*E+Cb*B+Ct*T <= max_budget)
@constraint(m, (E <= target_EVs)



In [ ]:
optimize!(m)
objective_value(m)@show 

In [ ]:
@show shadow_price(G)
@show shadow_price(E)
@show shadow_price(B)
@show shadow_price(T)